# Assignment-1: Transformations and representations

Team Name: GSMF

Roll number: 2019102015 2019102031

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `Set Up` for detailed step-by-step instructions about the installation setup.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. 
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 11/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.


# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

# Set Up

We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. All assignments will be python based, hence familiarising yourself with Python is essential.


## Setting up Anaconda environment (Recommended)

1. Install Anaconda or Miniconda from [here](https://docs.conda.io/projects/conda/en/latest/user-guide/install/linux.html) depending on your requirements.
2. Now simply run `conda env create -f environment.yml` in the current folder to create an environment `mr_assignment1` (`environment.yml` can be found in `misc/`).
3. Activate it using `conda activate mr_assignment1`.

## Setting up Virtual environment using venv

You can also set up a virtual environment using venv

1. Run `sudo apt-get install python3-venv` from command line.
2. `python3 -m venv ~/virtual_env/mr_assignment1`. (you can set the environment path to anything)
3. `source ~/virtual_env/mr_assignment1/bin/activate`
4. `pip3 install -r requirements.txt` from the current folder (`requirements.txt` can be found in `misc/`).

In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. Getting started with Open3D

Open3D is an open-source library that deals with 3D data, such as point clouds, mesh. We'll be using Open3D frequently as we work with point clouds. Let's start with something simple:

<img src="misc/bunny.jpg" alt="drawing" width="200"/>

1. Read the Stanford Bunny file (in `data/`) given to you and visualise it using Open3D.
2. Convert the mesh to a point cloud and change the colour of points.
3. Set a predefined viewing angle (using Open3D) for visualization and display the axes while plotting.
4. Scale, Transform, and Rotate the rabbit (visualise after each step).
5. Save the point cloud as bunny.pcd.

In [48]:
def custom_visualize(geometry_list,front,lookat,up):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    for i in geometry_list:
        vis.add_geometry(i)
    o3d.visualization.ViewControl.set_front(vis.get_view_control(), front)
    o3d.visualization.ViewControl.set_lookat(vis.get_view_control(), lookat)
    o3d.visualization.ViewControl.set_up(vis.get_view_control(), up)
    #o3d.visualization.ViewControl.set_zoom(vis.get_view_control(), 0.8)
    vis.run()
    vis.destroy_window()
    
    
###read and visualize mesh###
bunny_mesh = o3d.io.read_triangle_mesh("./data/bunny.ply")
bunny_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([bunny_mesh])

###convert mesh to point cloud and change colour###
num_points = 15000
bunny_pcd = bunny_mesh.sample_points_poisson_disk(num_points)
color = np.zeros((num_points,3))
color[:] = [0,0,0]
bunny_pcd.colors = o3d.utility.Vector3dVector(color)

###predefined viewing angle and display axes###
cf = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01,origin=bunny_pcd.get_center()-[0.1,0.1,0])
front = [ -0.17787558071731241, 0.19847101653089066, 0.96383065596694628 ]
lookat = [ -0.033225362169859066, 0.090336708898100304, -0.0014016149085554139 ]
up = [ -0.021124395343669902, 0.97845413353141153, -0.20538078901557275 ]
custom_visualize([cf,bunny_pcd],front=front,lookat=lookat,up=up)

###Scale###
bunny_pcd.scale(scale=1.5,center=bunny_pcd.get_center())
custom_visualize([cf,bunny_pcd],front=front,lookat=lookat,up=up)

###Transform###
T = np.array([ [ 0.933 , 0.067 , 0.354 , 0 ] , [ 0.067 , 0.933 , -0.354 , 0 ] , [ -0.354 , 0.354 , 0.866 , 0 ] , [ 0 , 0 , 0 , 1 ] ],np.float64)
bunny_pcd.transform(T)
custom_visualize([cf,bunny_pcd],front=front,lookat=lookat,up=up)

###Translate###
bunny_pcd.translate(translation=[0.1,-0.1,0],relative=True)
custom_visualize([cf,bunny_pcd],front=front,lookat=lookat,up=up)

###Rotate###
R = np.array([ [ np.cos(np.pi/4) , -np.sin(np.pi/4) , 0 ] , [ np.sin(np.pi/4) , np.cos(np.pi/4) , 0 ] , [ 0 , 0 , 1 ] ],np.float64)
bunny_pcd.rotate(R,center=bunny_pcd.get_center())
custom_visualize([cf,bunny_pcd],front=front,lookat=lookat,up=up)

###save as bunny.pcd###
o3d.io.write_point_cloud("./data/bunny.pcd",bunny_pcd)

True

# 2. Transformations and representations

## a) Euler angles
1. Write a function that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z)

2. Solve for angles using ```fsolve from scipy``` for three initializations of your choice and compare.
$$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] 
$$

$$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] 
$$

3. What is a Gimbal lock? 

4. Show an example where a Gimbal lock occurs and visualize the Gimbal lock on the given bunny point cloud. You have to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*


In [2]:
import math
import scipy

def get_rotation_matrix(a,b,c):
    rotation_matrix = np.array([[np.cos(a)*np.cos(b),np.cos(a)*np.sin(b)*np.sin(c) - np.sin(a)*np.cos(c) , np.cos(a)*np.sin(b)*np.cos(c) + np.sin(a)*np.sin(c)],
                                [np.sin(a)*np.cos(b),np.sin(a)*np.sin(b)*np.sin(c) + np.cos(a)*np.cos(c) , np.sin(a)*np.sin(b)*np.cos(c) - np.cos(a)*np.sin(c)],
                                [-np.sin(b),np.cos(b)*np.sin(c), np.cos(b)*np.cos(c)]])
    return rotation_matrix

def euclidean_dist(p1,p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2 )

def func(x):
    global rmat_ideal
    rmat = get_rotation_matrix(x[0],x[1],x[2])
    points = np.array([[1,1,1],[1,0,1],[1,1,0]])
    ideal = np.matmul(rmat_ideal,points.T)
    expected = np.matmul(rmat,points.T)
    
    return [euclidean_dist(ideal[:,0],expected[:,0]),
            euclidean_dist(ideal[:,1],expected[:,1]),
            euclidean_dist(ideal[:,2],expected[:,2])]
    
    

rmat_ideal = np.array([[0.26200263 , -0.19674724 , 0.944799],
                       [0.21984631 , 0.96542533 , 0.14007684 ],
                       [-0.93969262 , 0.17101007 , 0.29619813]])

print("Matrix 1: ")
for i in range(3):
    init = np.random.randn(3)
    root = scipy.optimize.fsolve(func,init)
    print(f"trial : {i} with initiallization {init[0],init[1],init[2]}")
    print(get_rotation_matrix(root[0],root[1],root[2]))


rmat_ideal = np.array([[0 , -0.173648178 ,  0.984807753],
                       [0 , 0.984807753 , 0.173648178  ],
                       [-1,0,0]])

print("Matrix 2: ")
for i in range(3):
    init = np.random.randn(3)
    root = scipy.optimize.fsolve(func,init)
    print(f"trial : {i} with initiallization {init[0],init[1],init[2]}")
    print(get_rotation_matrix(root[0],root[1],root[2]))



Matrix 1: 
trial : 0 with initiallization (-0.6888907088953038, -2.6249987040193643, 0.6643231075971943)
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.2198463   0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]
trial : 1 with initiallization (-0.6044546926829781, -0.6014019096416005, -0.0716913439142179)
[[ 0.36753335 -0.26121792  0.89257181]
 [ 0.10646226  0.96525215  0.23865052]
 [-0.92389665  0.00731319  0.38257221]]
trial : 2 with initiallization (-0.32122772634943947, 2.195270206939449, -1.1802304570530284)
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101008  0.29619813]]
Matrix 2: 
trial : 0 with initiallization (1.6225315838506342, -1.176483569463739, 1.079689606484006)
[[-1.24977367e-09 -1.73648176e-01  9.84807753e-01]
 [ 5.29878775e-10  9.84807753e-01  1.73648176e-01]
 [-1.00000000e+00  7.38849644e-10 -1.13877432e-09]]
trial : 1 with initiallization (1.9564177928629554, -0.801649147388966, 1.6977023727153004)
[[-4

/home/god-particle/virtual_env/mr_assignment1/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/god-particle/virtual_env/mr_assignment1/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [3]:
pi = math.pi
get_rotation_matrix(pi/18,0,0)

array([[ 0.98480775, -0.17364818,  0.        ],
       [ 0.17364818,  0.98480775,  0.        ],
       [-0.        ,  0.        ,  1.        ]])

# Solution
### Gimbal lock 
Gimbal lock is a phenomenon which occurs when we lose a degree of freedom when we rotate a rigid body in 3D space in a particular manner.This occurs when axes of two initially orthogonal axes coincides and the rotation about any of the two axes are equivalent. This leads to firstly, loss of a degree of freedom and secondly, ambiguity in determining the euler angles. 
This occurs ,for  example , in (X-Y-Z) euler angles when we rotate about Y axis of local frame by 90$^\circ$ . This leads to the rotation matrix becoming 
$$R(\alpha, \beta ,\gamma)=\left[\begin{array}{rrr} 0 & 0 & 1
\\ sin(\alpha+\gamma) & cos(\alpha+\gamma) & 0 \\
   -cos(\alpha+\gamma) & sin(\alpha+\gamma) & 0\end{array}\right] 
$$

From this matrix we can only get information about sum of $ \alpha$ and $\gamma $ and not about their absolute values. This creates ambiguity about exact values.

## b) Quaternions

1. What makes Quaternions popular in graphics? 
2. Convert a rotation matrix to quaternion and vice versa. Do not use inbuilt libraries for this question.
3. Perform matrix multiplication of two $\mathcal{R}_{3 \times 3}$ rotation matrices and perform the same transformation in the quaternion space. Verify if the final transformation obtained in both the cases are the same.
4. Try to interpolate any 3D model (cube / bunny / not sphere obviously!!) between two rotation matrices and visualize!

The above questions require you to **code your own functions** and **only verify** using inbuilt functions.

# Solution:
### Quaternions
Quaternions are a 4D representation of rotations in 3D space. It is popular in graphics because:
1. Since they are 4D representation of 3D rotations, they are sufficient enough to represent the rotations without any ambiguity. Quaternion Representation is not prone to Gimbal locks because of this feature.
2. Although it is difficult to imagize 4D representations and their projections/effects in warping 3D spaces, It is very easy in computational sense. It is similar to complex number operations and is free from matrix operations.
3. Since each quaternion requires only 4 real numbers to be representated, it requires less memory and is computationally efficient.

In [47]:
def quaternion_to_rotation_matrix(Q):
    q0 = Q[0]
    q1 = Q[1]
    q2 = Q[2]
    q3 = Q[3]

    r00 = 1 - 2 * (q2 * q2 + q3 * q3) 
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)

    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 1 - 2 * (q1 * q1 + q3 * q3) 
    r12 = 2 * (q2 * q3 - q0 * q1)

    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 1 - 2 * (q1 * q1 + q2 * q2) 
     
    rotation_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rotation_matrix
def rotation_matrix_to_quaternion(R):
    r00 = R[0][0]
    r01 = R[0][1]
    r02 = R[0][2]
    r10 = R[1][0]
    r11 = R[1][1]
    r12 = R[1][2]
    r20 = R[2][0]
    r21 = R[2][1]
    r22 = R[2][2]
    
    r = np.math.sqrt(1.0+r00+r11+r22)*0.5
    i = (r21-r12)/(4*r)
    j = (r02-r20)/(4*r)
    k = (r10-r01)/(4*r)
    
    return np.array([r,i,j,k])

def quaternion_mult(Q1,Q2):
#     (w_1+x_1*i+y_1*j+z_1*k)(w_2+x_2*i+y_2*j+z_2*k)
    w1,x1,y1,z1 = Q1[0],Q1[1],Q1[2],Q1[3]
    w2,x2,y2,z2 = Q2[0],Q2[1],Q2[2],Q2[3]
    W_mult = w1*w2 -x1*x2 -y1*y2 -z1*z2
    X_mult = w1*x2 + w2*x1 +y1*z2 -y2*z1
    Y_mult = w1*y2 + w2*y1 + z1*x2 - x1*z2
    Z_mult = w1*z2 + w2*z1 + x1*y2 - x2*y1
    
    return np.array([W_mult,X_mult,Y_mult,Z_mult])

alpha1 = (math.pi)/3 
beta1 = (math.pi)/4 
gamma1 = (math.pi)/6
alpha2 = math.pi/2
beta2 = 3*math.pi/4 
gamma2 = math.pi/3
R1 = get_rotation_matrix(alpha1,beta1,gamma1)
R2 = get_rotation_matrix(alpha2,beta2,gamma2)
Q1 = rotation_matrix_to_quaternion(R1)
Q2 = rotation_matrix_to_quaternion(R2)
Rmult = np.matmul(R1,R2)
Q_mult = quaternion_mult(Q1,Q2)
print(Rmult)
print(quaternion_to_rotation_matrix(Q_mult))


[[-0.11736248 -0.98046789 -0.15782511]
 [-0.72091587 -0.02518759  0.69256472]
 [-0.6830127   0.19505974 -0.70387879]]
[[-0.11736248 -0.98046789 -0.15782511]
 [-0.72091587 -0.02518759  0.69256472]
 [-0.6830127   0.19505974 -0.70387879]]


## c) Exponential maps (Bonus)

1. What is the idea behind exponential map representation of rotation matrices?
2. Perform matrix exponentiation and obtain the rotation matrix to rotate a vector $P$ around $\omega$ for $\theta$ seconds.
$$
\omega = \begin{bmatrix}2 \\ 1 \\ 15 \end{bmatrix}
$$

$$
\theta = 4.1364
$$

3. Compute the logarithmic map (SO(3) to so(3)) of the rotation matrix to obtain the rotation vector and the angle of rotation
$$
\begin{bmatrix}
0.1 &  -0.9487 & 0.3 \\
0.9487 & 0.  & -0.3162 \\
0.3   &  0.3162  & 0.9 
\end{bmatrix}
$$
You can use inbuilt libraries **only to verify** your results.

# 3. Data representations

## a) Octomaps

1. Why is an Octomap memory efficient?
2. When do we update an Octomap and why?
3. When would you likely use an octomap instead of a point cloud?
 

## Solution:
#### Why octomaps are memory efficient
Octomaps are generated by using an efficient datastructure caled as Octrees. An octree is an heirarchical datastructure used for spatial subdivision in computer vision. At each node we can either divide the space into eight smaller voxels if the current bigger voxel isnt  completely unoccupied or we can stop dividing if it is completely unoccupied . We recursively subdivide unless we reach minimum sized voxel ,ie, the resolution. 

Since we are not storing each and every cell as individual resolution sized voxels and rather we are only dividing a bigger cell if it is not completely unoccupied, this leads to saving a lot of memory which makes them memory efficient.

#### When and why do we update an octomap

We update an octomap when we recieve information from the sensors. Since the sensors are erroneous, we will always have a probability whether the laser was reflected or it travelled through the reflecting volume. When we move through the space, we need to scan the surroundings occasionally , which leads to updating the octomaps wherever neccesary.
Each time we have a scan using sensors, we have current measurements and prior probability that the volume was occupied upto previous readings . Using this , we get conditional probabilty of occupancy of the voxel. If it is greater than a particular threshold, we say that the voxel is occupied. This leads to update in the octomaps.

#### When do we use octomap instead of point cloud.

We use octomaps for problems involving trajectory planing ,ex, path planing for drones. Planing paths using octomaps are computationally efficient and also easier problem . Using point clouds for such problems can be very difficult to solve and may also require lot of memory.

## b) Signed Distance Functions

1. How do we determine object surfaces using SDF?
2. How do we aggregate views from multiple cameras? (just a general overview is fine)
3. Which preserves details better? Voxels or SDF? Why?
4. What’s an advantage of SDF over a point cloud?


# References and Resources

1. Gimbal locks and quaternions: https://youtu.be/YF5ZUlKxSgE
2. Exponential map: 
    1. 3 Blue 1 Brown: https://youtu.be/O85OWBJ2ayo
    2. Northwestern Robotics: https://youtu.be/v_KBHaG0mas
3. Bunny ply is taken from: http://graphics.im.ntu.edu.tw/~robin/courses/cg03/model/